In [92]:
### Iteration 3,4: topic modeling with removal of irrelevant keywords

In [16]:
category = 5

In [17]:
import pandas as pd
import numpy as np
import preprocessor as p
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')

df = pd.read_csv(f"/mnt/smdata/twitter/VAMoS/finalData/ALLcat{category}.csv", engine='python')
print("Raw Tweets scraped: " + str(df.shape[0]))
#print(df.text)

Raw Tweets scraped: 38311


In [18]:
df['text'] = df['text'].apply(str.lower)
#print(df.text)

df = df.drop_duplicates(subset=['text'])
print("after removing duplicate tweets: " + str(df.shape[0]))

after removing duplicate tweets: 31948


In [19]:
# remove irrelevant topic keywords per category
remove1 = ['pea','seed','bean','podcast','radio','episode','listen','iphone','apple','spotify','whale','dolphin', 'youtube','video','audio','show','recording','live','song','check','coffee','capsule','nespresso','arsenal','world','chelsea','football','rugby','soccer','review','invest','podium','instagram','elon','linkedin','delete','escape','space','bath','earth','mars','ocean','tribunal','judicial','trial','just','venez','president','colombia','bugs','shirt','wave','tide','business','demand','holiday','tickets','podcasts','politic','election','education','school','missile','ukraine','twins','birth','airpod','sleep','race','wrestling','trump','media','crypto','bean','nfl','season','tonight','baylor','postgame','game','plays','nba','headphones','earphones','air','mentions','enjoyed','thank','taylor','swift','kravitz','twitter','tweet','battery','paintball','kubernetes','devops','unsubbed','tassimo','entire','card','followers','discount','relx','inf','team','trade','guest','forward','happy','subscribe']
remove2 = ['seed','ball','perth','australia','booking','sydney','melbourne','plant','traffic','parking','york','onlyfan','camgirl','bank','dog','pet','podcast','trend','nfts','crypto','grow','nairobi','whatsapp','kenya','horny','porn','twitter','election','republi','vote','senate','democrat','russia','ukraine','pain','medical','opioid','patient','metro','bus','residential','route','street','stock','twitter','melatonin','sleep','night','insomnia','media','platform','tax','government','thailand','asia','bankok','california','green','giant','jerry','vicks','chest','nose','rub','anxiety','disorder','earth','atmosphere','water','almond','cook','spinach','biden','president','prison','citizen','holiday','christmas','patriot','navy','christmas','jersey','johannesburg','faded','lounges','arroz','tren','crime','driver','art','cancer','africans','never','friendly','oveja','train','foreigners','roadside','painting','breast','massage','aceite','pepper','mongodb','workstation','wax','trail','company','investing','german','europe','blockchain']
remove3 = ['ukraine', 'congratulations','thank','winners','please','much','great']
remove4 = ['ukraine']
remove5 = ['ukraine','lady', 'episode','pronouns','zodiac','tattoo','height','radio','podcast','music','radio','twitter','onlyfan','tweet','listen','memory6','distract','mirror','want','would','dont','trump','republican','democrat','election','president','congress','fav','nowplaying','girls','alarms','god','crack','sleep','height','pop','women'	,'alarm'	,'church'	,'russian',	'cmere'	,'bed','tattoos',	'song',	'men',	'test'	,'satan',	'russia',	'crackhead',	'nap','piercings',	'listen'	,'ladies'	,'detectors',	'lord',	'kyiv',	'dontcha','zodiac'	,'feat'	,'girlfriend'	,'fire'	,'shall',	'moscow',	'offer',	'insomnia','pronouns'	,'tyga',	'mcstan',	'carbon'	,'bible'	,'donetsk',	'echelon',	'ez','playing'	,'marry'	,'monoxide'	,'holy','aiko'	,'drink',	'home',	'torment',	'explosion','pop',	'shop'	,'gravedigger', 'woo'	,'suspects',	'memory', 'killed',	'rick',	'years'	,'robbery','choked',	'watching' ,'somewhere',	'movie', 'castle', 'headlight', 'compatible', 'machine', 'hammer',	'sunrise' ,'boston', 'absolute',	'arctic' ,'train'	,'black', 'palestine', 'derail' ,'minute' ,'travelling', 'discount' ,'battery'	,'balloon'	,'fire',	'plane',	'christmas',	'art','igbo', 'batteries',	'bomb',	'theres',	'flight',	'year',	'painting',	'dey' ,'photoelectric',	'grenade',	'wildfires',	'airport'	,'holiday'	,'paint',	'werey','beep',	'fighter'	,'typically'	,'drone',	'fryd'	,'sketch',	'seun','game','player','football','eagles','cowboys','turkey','cheese','pork','episode','season','recap','seaweed','water','frozen','ocean','weather','humidity','temperature','democracy','politic','ballot','senator','screen','valentine']
remove6 = ['ukraine','podcast']
remove = [remove1,remove2,remove3,remove4,remove5,remove6]

df = df.loc[~df['text'].str.contains('|'.join(remove[category-1]))]
print("after removing irrelevant stuff: " + str(df.shape[0]))

after removing irrelevant stuff: 22739


In [20]:
#convert to list 
tweets = df.text.to_list()
print("total tweets = "+str(len(tweets)))

# remove URLs, mentions, emojis, RT etc.
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.RESERVED, p.OPT.SMILEY, p.OPT.NUMBER)

cleaned_tweets = []
for i in tweets:
    ct = p.clean(i)
    cleaned_tweets.append(ct)

seen = set()
unique_tweets = []
for item in cleaned_tweets:
    if item not in seen:
        seen.add(item)
        unique_tweets.append(item)

print("total unique preprocessed tweets = "+str(len(unique_tweets)))

"""
k=1
for tweet in unique_tweets:
    print(k)
    print(tweet)
    k=k+1
"""

total tweets = 22739
total unique preprocessed tweets = 22454


'\nk=1\nfor tweet in unique_tweets:\n    print(k)\n    print(tweet)\n    k=k+1\n'

In [21]:
# language detection
from langdetect import detect

eng_esp = []
for tweet in unique_tweets:
    try:
        if len(tweet)>0 and (detect(tweet)=='en' or detect(tweet)=='es'):
            eng_esp.append(tweet)
    except:
        continue

print("eng/esp tweets = "+str(len(eng_esp)))

eng/esp tweets = 4621


In [22]:
#geolocation using mordecai that uses docker in the background
# -> sudo docker container ls
# -> sudo docker stop <...>
# ->  sudo docker .... see mordecai github repo
import spacy
nlp = spacy.load("/home/ubuntu/.local/lib/python3.8/site-packages/en_core_web_lg/en_core_web_lg-2.3.1/")

from mordecai import Geoparser
geo = Geoparser()

state = []
country = []

for i in range(0,len(eng_esp)):
    if i%1000 == 0:
        print("<------------------"+str(i)+"------------------>")
    a = geo.geoparse(eng_esp[i])
    for j in range(0,len(a)):
        if a[j]['country_conf'] > 0.75:
            #print(a)
            print(str(a[j]["geo"]["admin1"]) + " : " + str(a[j]["geo"]["country_code3"]))
            state.append(a[j]["geo"]["admin1"])
            country.append(a[j]["geo"]["country_code3"])

sk = [('state',state),('country',country)]
data = dict(sk)
df = pd.DataFrame(data)
df.to_csv(f"topicModels/iter4_GEO_cat{category}.csv",index=False)
print("saved location data to csv")
            

'\n#geolocation using mordecai that uses docker in the background\n# -> sudo docker container ls\n# -> sudo docker stop <...>\n# ->  sudo docker .... see mordecai github repo\nimport spacy\nnlp = spacy.load("/home/ubuntu/.local/lib/python3.8/site-packages/en_core_web_lg/en_core_web_lg-2.3.1/")\n\nfrom mordecai import Geoparser\ngeo = Geoparser()\n\nstate = []\ncountry = []\n\nfor i in range(0,len(eng_esp)):\n    if i%1000 == 0:\n        print("<------------------"+str(i)+"------------------>")\n    a = geo.geoparse(eng_esp[i])\n    for j in range(0,len(a)):\n        if a[j][\'country_conf\'] > 0.75:\n            #print(a)\n            print(str(a[j]["geo"]["admin1"]) + " : " + str(a[j]["geo"]["country_code3"]))\n            state.append(a[j]["geo"]["admin1"])\n            country.append(a[j]["geo"]["country_code3"])\n\nsk = [(\'state\',state),(\'country\',country)]\ndata = dict(sk)\ndf = pd.DataFrame(data)\ndf.to_csv(f"topicModels/eng_esp_GEO_cat{category}.csv",index=False)\nprint("sav

In [23]:
#removing stopwords
all_languages = ['arabic', 'azerbaijani', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek','hungarian', 'indonesian', 
 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']

enes = ['english', 'spanish']

stops = set(stopwords.words(all_languages))
print("total stopwords in all languages = "+ str(len(list(stops))))

stops_enes = set(stopwords.words(enes))
print("total stopwords in english/espanol = "+ str(len(list(stops_enes))))

vectorizer_model = CountVectorizer(analyzer = 'word', stop_words=list(stops_enes))

total stopwords in all languages = 6715
total stopwords in english/espanol = 485


In [24]:
topic_model = BERTopic(nr_topics="auto", vectorizer_model=vectorizer_model, language="multilingual")

topics, probabilities = topic_model.fit_transform(eng_esp)

tot_topics = len(topic_model.get_topics()) - 1
print("total topics = " + str(tot_topics))
topic_model.get_topic_freq().head(11)

total topics = 58


,Topic,Count
0,-1,2332
1,0,262
2,1,167
3,2,140
4,3,74
5,4,72
6,5,69
7,6,63
8,7,58
9,8,54


In [25]:
topic_model.visualize_barchart(top_n_topics=12,n_words=10,title=f"Cat {category} (# eng/esp tweets = {len(eng_esp)}) : Topic word scores (total = {tot_topics})")

In [26]:
topic_model.save(f"/mnt/smdata/twitter/VAMoS/topicModels/iter4_enes_cat{category}_model")
print("model saved")
df = topic_model.get_document_info(eng_esp)
df.to_csv(f"topicModels/iter4_tweets_per_topic_cat{category}_enes.csv")
print("doc info saved")

model saved
doc info saved


In [27]:
a = topic_model.get_topics()
n = len(a) - 1
print("total topics = " + str(n))

total topics = 58


In [28]:
# top topics
if n > 40:
    n = int(0.75*n)
    print(n)

sk = []
for i in range(0,n):
    x = []
    for j in range(0,10):
        x.append(a[i][j][0])
    sk.append((f'topic{i}', x))
data = dict(sk)
df = pd.DataFrame(data)
df.to_csv(f"topicModels/iter4_majorTopics_cat{category}_enes.csv",index=False)
print("saved topics to csv")

43
saved topics to csv


In [29]:
topic_model.get_topics()

{-1: [('pod', 0.01576411906470047),
  ('rt', 0.015348000816621875),
  ('new', 0.007212801088746513),
  ('amp', 0.006792690511150511),
  ('one', 0.006650027636323757),
  ('get', 0.006196179771871554),
  ('like', 0.005626946461504717),
  ('today', 0.005578986131372087),
  ('great', 0.005341832349502925),
  ('back', 0.005077311486558699)],
 0: [('hes', 0.02442767717870198),
  ('guy', 0.02160610669171632),
  ('pod', 0.01571557195966138),
  ('said', 0.013401052513052016),
  ('like', 0.012779660765918596),
  ('get', 0.011965004730403024),
  ('think', 0.011183671242365086),
  ('week', 0.010803064803148733),
  ('would', 0.01077081644502176),
  ('way', 0.0105293984031123)],
 1: [('si', 0.022810290321117227),
  ('ser', 0.021208571641764637),
  ('rt', 0.0192770578472015),
  ('ms', 0.01699620279983083),
  ('creo', 0.01689007057169854),
  ('dios', 0.01583510912563956),
  ('pod', 0.015402845572021942),
  ('tan', 0.01531489019321888),
  ('dejar', 0.014477203347170177),
  ('siempre', 0.014368755762766

In [30]:
#reduce the number of topics after training the model. This is a great option if retraining the model will take many hours
#new_topics, new_probs = topic_model.reduce_topics(cleaned_tweets, nr_topics=250)